Analyse des données

Préparation des données avec pandas


In [19]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import os
import sys
import pycountry

sys.path.append(os.path.abspath(".."))

pd.set_option("display.float_format", lambda x: f"{x :.2f}")
pd.set_option("display.max_columns", None)
from get_dataframes import GetDataframes

from tools import import_config

from plotting import (
    movies_by_decades,
    movies_by_genres,
    movies_by_duration_boxplot,
    movies_by_country,
    movies_duration_by_decades_boxplot,
    movies_top_x,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- On importe la dataframe "title_basics" qui contient les infos pour tout les films de la base de données.
- On enlève tout les films à caractère pornographique de notre liste.
- On nettoie les /N et les transforme en 0.
- On change les types de données des colonnes (float en int, etc...)
- On merge la dataframe "title_basics" avec "tmdb_full" pour obtenir des infos supplémentaires sur chaque film.
- On supprime les colonnes inutiles.
- On supprime les lignes pour lesquelles on a des valeurs manquantes suite au merging.
- On transforme les données dans les colonnes ayant plusieurs valeurs et les on les transforme en listes.
- On importe les dataframes "name_basics", "title_akas" et "title_principals".
- On merge la dataframe "title_akas" avec "tmdb_full".
- On filtre les films ayant eu une adaptaion en FR uniquement.
- On filtre seulement les films qui sont "Released".
- On nettoie "name_basics" comme précédement.
- On merge "name_basics", "title_principals" et on trie les acteurs, directeurs et autres dans un autre dataframe.
- On merge chaque dataframe (acteurs, directeurs, etc...) avec la liste des films qui étaient déjà merged avec tmdb.

In [20]:
config = import_config(add=True)
datas = GetDataframes(config)
link = "movies"
df = datas.get_dataframes(link)

2023-11-21 14:13:31 INFO     TSV files already exist.
2023-11-21 14:13:31 INFO     Parquet loaded ! Importing movies...
2023-11-21 14:13:31 INFO     Dataframe movies ready to use!


- On créée une visualisation pour voir la distribution des notes moyennes de chaque film, le nombre de films par décénnie et le total de votes par décénnie.

In [21]:
def iso_to_country_name(iso_codes):
    country_names = []
    for code in iso_codes:
        country = pycountry.countries.get(alpha_2=code)
        if country:
            country_names.append(country.name)
        else:
            country_names.append("Inconnu")
    return ", ".join(country_names)


df["production_countries"] = df["production_countries"].apply(
    iso_to_country_name
)

In [22]:
df[df["titre_duree"] == df["titre_duree"].max()]

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,region,cuts
44929,tt3854496,Ambiancé,2020,43200,"[Fantasy, Drama]",3.40,116,sv,Ambiancé,1.82,Sweden,0,[sv],Released,FR,>2020


In [23]:
figs = movies_by_decades(df)

for fig in figs:
    fig.show()

- On cherche quel genre de films est le plus répandu dans le dataset filtré

In [24]:
movies_by_genres(df)

- On peut voir que le nombre de films en dessous des années 1960 est très bas et nous décidons de ne pas les garder.
- Les films ayant moins de 5 en note moyenne ne nous intéressent pas non plus.
- Et les films ayant reçu trop peu de votes sont aussi enlevés.

In [25]:
condi = (
    (df["titre_date_sortie"] >= 1960)
    & (df["rating_avg"] >= 6.3)
    & (df["rating_votes"] >= 5004)
)

df = df[condi].reset_index(drop="index")

- Ici nous filtrons les genres qui n'ont que peu d'interet pour le cinéma.

In [26]:
genres_ = ["Music", "Documentary", "Reality-TV", "News"]
df = df[
    df["titre_genres"].apply(lambda x: all(g not in x for g in genres_))
]

In [27]:
movies_by_genres(df)

- On peut désormais ré-afficher la distribution des notes moyennes, le total des films par décénnie et le nombre moyen des votes par décénnies

In [28]:
figs = movies_by_decades(df)

for fig in figs:
    fig.show()

- On cherche des informations sur la durée des films en minutes

In [29]:
movies_by_duration_boxplot(df)

- On constate qu'il y a beaucoup trop de films avec des durées bien trop longues ainsi que des films trop courts qui devraient être considéré plutot comme des court-métrages.
- On enlève donc les films en dessous de 62 minutes et au dessus de 210 minutes (3h30)

In [30]:
condi = ~((df["titre_duree"] < 62) | (df["titre_duree"] > 210))
df = df[condi].reset_index(drop="index")

movies_by_duration_boxplot(df)

- On cherche les 10 premiers pays desquels proviennent les films du dataset.

In [31]:
movies_by_country(df)

- On cherche si il y a une augmentation ou diminution de la durée des films dans le temps.

In [32]:
movies_duration_by_decades_boxplot(df)

In [33]:
# df[((df["titre_duree"] > 170) & (df["titre_date_sortie"] > 2020))]

- On cherche les 10 films avec le plus de votes sans compter les documentaires, concerts et films d'animations.

In [34]:
movies_top_x(df, top=10)

In [35]:
# #  World record of the longest documentary
# # source :https://fr.wikipedia.org/wiki/Ambianc%C3%A9
# longest = actor[actor["titre_duree"] == 43200]
# n = longest[["titre_str", "titre_duree"]].iloc[0]
# heure = n["titre_duree"] // 60
# jours = heure // 24
# print(f"Le film le plus long est {n['titre_str']} d'une durée de {heure} heures soit {jours} jours")
# # All time median for duration
# print(f"All time duration median : {round(actor['titre_duree'].median())} minutes")